In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import IPython
from IPython.display import display, HTML, Image
import qgrid
import myql
from yahoo_oauth import OAuth1
from myql import MYQL
import xml.etree.ElementTree as ET
import pymysql.cursors
import warnings
warnings.filterwarnings('ignore')

class NHL_Info:
    
    oauth = OAuth1(None, None, from_file='../oauth1.json') # only consumer_key and consumer_secret are required.
    guid = oauth.guid
    consumer_key = oauth.consumer_key
    consumer_secret = oauth.consumer_secret
    session_handle = oauth.session_handle
    access_token_secret = oauth.access_token_secret
    yql = MYQL(format='xml', oauth=oauth)

    league_key = 'nhl.l.22070'
    my_team_key = '363.l.22070.t.9'

    NSMAP = {'yh':'http://fantasysports.yahooapis.com/fantasy/v2/base.rng'}
    
    pd.set_option('precision', 2)   
    # relative location of fantasy league relevant statistical categories in player statistics array
    skater_stat_ids = [1,2,3,4,6,12,13,14,15,17]    
    # qgrid options for nice table layout
    grid_opts = {
    'rowHeight': 35,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'defaultColumnWidth': 140,
    'asyncEditorLoading': True,
    'topPanelHeight': 40
    }   
    no_NHL_teams = 30  
    team_abbr = ['ANA','ARI','BOS','BUF','CGY','CAR','CHI','COL','COB','DAL','DET','EDM','FLA','LOS','MIN',
                 'MON','NAS','NJD','NYI','NYR','OTT','PHI','PIT','SAN','STL','TAM','TOR','VAN','WAS','WPG']
    
    def __init__(self, week):
        self.week = week
        
    def set_fantasy_league_info(self, df):
        self.df_league = df

    def get_fantasy_league_info(self):
        
        if not self.oauth.token_is_valid():
            self.oauth.refresh_access_token()

        response = self.yql.raw_query('select * from fantasysports.leagues.standings where league_key in ("'+self.league_key+'")') # Deal with the response
        league_tree = ET.fromstring(response.text)

        # get the league info
        league = []
        for team in league_tree.findall('.//yh:team', namespaces=self.NSMAP):
            name = team.find('.//yh:name', namespaces=self.NSMAP).text
            rank = int(team.find('.//yh:rank', namespaces=self.NSMAP).text)
            wins = int(team.find('.//yh:wins', namespaces=self.NSMAP).text)
            losses = int(team.find('.//yh:losses', namespaces=self.NSMAP).text)
            ties = int(team.find('.//yh:ties', namespaces=self.NSMAP).text)
            wlt = '{}-{}-{}'.format(wins,losses,ties)
            points = float(team.find('.//yh:points_for', namespaces=self.NSMAP).text)
            team_key = team.find('.//yh:team_key', namespaces=self.NSMAP).text
            email = team.find('.//yh:email', namespaces=self.NSMAP).text
            draft_position = team.find('.//yh:draft_position', namespaces=self.NSMAP).text
            moves = team.find('.//yh:number_of_moves', namespaces=self.NSMAP).text
            trades = team.find('.//yh:number_of_trades', namespaces=self.NSMAP).text
            league.append({'name':name, 'rank':rank, 'w-l-t':wlt, 'points':points, 'team_key':team_key,
                           'draft_pos':draft_position, 'moves':moves, 'trades':trades})
        df_league = pd.DataFrame(league, columns=['name', 'rank', 'w-l-t', 'points', 'moves', 'trades', 
                                                  'draft_pos', 'team_key'])
        self.set_fantasy_league_info(df_league)
        return df_league

    def get_fantasy_roster_info(self, team_key):
        
        if not self.oauth.token_is_valid():
            self.oauth.refresh_access_token()
            
        response = self.yql.raw_query('select * from fantasysports.teams.roster where team_key in ("'+team_key+'")')
        roster_tree = ET.fromstring(response.text)
        players = []
        for player in roster_tree.findall('.//yh:player', namespaces=self.NSMAP):
            name = player.find('.//yh:full', namespaces=self.NSMAP).text
            players.append(name)
            
        return players
    
    '''
    Setter for NHL teams in the league, including mapping the full names to the respective abbreviations 
    @params [self]           
    @returns [team_games]    a dictionary of {team,no_games} key value pairs
    ''' 
    def set_NHL_teams(self, teams):
        self.teams = sorted(teams)
        self.teams_dict = dict(zip(self.teams, self.team_abbr))

    '''
    Scrapes Yahoo to find the games per week per NHL team 
    @params [self]           
    @returns [team_games]    a dictionary of {team,no_games} key value pairs
    ''' 
    def get_games_per_week(self):

        # make connection to Yahoo games per week page
        page = requests.get("https://hockey.fantasysports.yahoo.com/hockey/team_games?week="+str(self.week)+"")
        soup = BeautifulSoup(page.content, 'html.parser')

        teams = []
        unparsed_teams = soup.select("td div a")
        # get the teams 
        for i in range(len(unparsed_teams)):
            teams.append(unparsed_teams[i].get_text())

        no_games = []
        unparsed_games = soup.find_all("td", class_="stat Tst-games")
        # get the number of games
        for i in range(len(unparsed_games)):
            no_games.append(unparsed_games[i].get_text())
            
        assert len(teams) == len(no_games), \
            "Length of teams: %r is not equal to length of no_games: %r." % (len(teams), len(no_games))

        # put them in a dictionary
        team_games = dict(zip(teams, no_games))
        # parse the results to get rid of unwanted teams
        team_games = { k:v for k,v in team_games.items() if "All-Stars" not in k }
        
        assert len(team_games) == self.no_NHL_teams, \
            "There are: %r teams in the NHL but team_games is of size: %r." % (self.no_NHL_teams, len(team_games))
            
        self.set_NHL_teams(team_games.keys())
        
        # replace the long team names with abbreviations
        team_games = {self.teams_dict[k]: v for k, v in team_games.items()}

        return team_games
    
    '''
    Writes the games per week dictionary to csv
    @params [self, dict]           
    @returns [team_games]    a dictionary of {team,no_games} key/value pairs
    ''' 
    def games_per_week_to_csv(self, dict):
        with open('nhl_games_per_week_'+str(self.week)+'.csv', 'wb') as f:
            w = csv.writer(f)
            w.writerows(dict.items())

    '''
    Scrapes Yahoo to find the YTD NHL player statistics and prints a table of the results
    @params [self]           
    ''' 
    def ytd_player_details(self):
        
        page = requests.get("https://ca.sports.yahoo.com/nhl/stats/byposition?pos=C,RW,LW,D")
        soup = BeautifulSoup(page.content, 'html.parser')
        
        unparsed_headers = soup.find_all("a", class_="ysptblhdrsts")
        headers = []
        # get the headers 
        for i in range(len(unparsed_headers)):
            headers.append(unparsed_headers[i].get_text())
        headers = [headers[i] for i in range(len(headers)) if i in self.skater_stat_ids]
        headers.remove('PPG')
        headers.remove('SHG')
        headers = [s.replace('PPA', 'PPP') for s in headers]
        headers = [s.replace('SHA', 'SHP') for s in headers]
        headers.append('Points')
        headers.append('Average_PPG')

        # get the player data
        unparsed_data = soup.find_all("td", class_="yspscores")
        data = []
        for i in range(len(unparsed_data)):
            data.append(unparsed_data[i].get_text())
        # clear unwanted spaces
        data = [s.strip() for s in data]
        data = list(filter(None, data))
        # split each player into their own list, all of which are appended to a new list
        player_data = [data[x:x+19] for x in range(0, len(data),19)]
        player_names = [data[i][:] for i in range(0, len(data),19)]
        relevant_player_data = []
        relevant_player_data_dict = {}
        points = None
        average_ppg = None
        for player in player_data:
            for i in range(len(player)):
                name = player[0]
                team = player[1]
                gp = player[2]
                g = player[3]
                a = player[4]
                pim = player[6]
                ppp = int(player[12]) + int(player[13])
                shp = int(player[14]) + int(player[15])
                sog = player[17]
                if i in self.skater_stat_ids:
                    if i == 12:
                        relevant_player_data.append(ppp)
                    elif i == 14:
                        relevant_player_data.append(shp)
                    elif i == 13 or i == 15:
                        pass
                    else:
                        relevant_player_data.append(player[i])
                points = ( int(g)*(3)+int(a)*(2)+int(pim)*(0.25)+int(ppp)*(0.5)+int(shp)*(1)+int(sog)*(0.5) )
                average_ppg = points/int(gp)
            relevant_player_data.append(points)
            relevant_player_data.append(average_ppg)
        
        final_relevant_player_data = [relevant_player_data[x:x+10] for x in range(0, len(relevant_player_data),10)] 
        df_player_data = pd.DataFrame(data=final_relevant_player_data, columns=headers, index=player_names)
        df_player_data.index.name = 'Name'
        qgrid.show_grid(df_player_data, remote_js=True, grid_options=self.grid_opts)
        return df_player_data
    
    '''
    Queries database to get past 3 seasons of player data
    @params [self]           
    @returns [df]    pandas dataframe of player stats with player name as the index
    ''' 
    def get_past_3_seasons_player_stats(self):         
        # Connect to the database
        connection = pymysql.connect(host='172.17.0.2',
                                     port=3306,
                                     user='root',
                                     password='',
                                     db='db_fhlo',
                                     cursorclass=pymysql.cursors.DictCursor)
        try:
            with connection.cursor() as cursor:
                # Read a single record
                sql = "SELECT * FROM `seasons_past`"
                cursor.execute(sql)
                df = pd.DataFrame(cursor.fetchall(), columns=['fname','lname','age','pos','gp_1314','gp_1415','gp_1516',
                                                              'pts_1314','pts_1415','pts_1516',])
                df["name"] = df["fname"] + ' ' + df["lname"]
                df.set_index(['name'], inplace=True)
                del df['fname']
                del df['lname']
                df["avg_gp"] = df[['gp_1314','gp_1415','gp_1516']].mean(axis=1)
                df["avg_pts"] = df[['pts_1314','pts_1415','pts_1516']].mean(axis=1)
                df["avg_ppg"] = df["avg_pts"]/df["avg_gp"]
                # rename the columns for aesthetics
                df = df.rename(columns = {'name':'Name', 'pos':'Position', 'age':'Age', 'gp_1314':'GP_1314',
                                          'gp_1415':'GP_1415', 'gp_1516':'GP_1516', 'avg_gp':'Average_GP_Past',
                                          'pts_1314':'Points_1314', 'pts_1415':'Points_1415', 'pts_1516':'Points_1516', 
                                          'avg_pts':'Average_Points_Past','avg_ppg':'Average_PPG_Past'})
        finally:
            connection.close()
            return df

    '''
    Pretty prints the key/value pairs of passed dictionary
    @params [self, dict]           
    ''' 
    def print_dict(self, dict):
        df_dict = pd.DataFrame.from_dict(data=dict, orient='index')
        df_dict.columns = ['Games_Per_Week']
        df_dict.index.name = 'Team'
        qgrid.show_grid(df_dict, remote_js=True, grid_options=self.grid_opts)
        return df_dict


[2017-01-25 23:17:38,444 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2017-01-25 23:17:38,445 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 3383.63316798
[2017-01-25 23:17:38,446 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [2]:
week_info = NHL_Info(15)

In [3]:
df_player_data = week_info.ytd_player_details()

In [4]:
games_per_team = week_info.get_games_per_week()
df_dict = week_info.print_dict(games_per_team)

In [5]:
df_league = week_info.get_fantasy_league_info()
# qgrid.show_grid(df_league, remote_js=True)

[2017-01-25 23:17:42,575 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 3387.76343393
[2017-01-25 23:17:42,576 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID
[2017-01-25 23:17:42,577 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 3387.7651329
[2017-01-25 23:17:42,577 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [6]:
my_players_list = week_info.get_fantasy_roster_info(week_info.my_team_key)

[2017-01-25 23:17:43,255 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 3388.43584085
[2017-01-25 23:17:43,256 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID
[2017-01-25 23:17:43,257 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 3388.44571495
[2017-01-25 23:17:43,258 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [7]:
df_my_players_list = pd.DataFrame(data=my_players_list)
df_player_data = df_player_data.loc[my_players_list]
df_player_data = df_player_data[['Team', 'GP', 'Points', 'Average_PPG']]
df_dict = df_dict.convert_objects(convert_numeric=True)
df_player_data = pd.merge(df_player_data,df_dict, right_on=df_dict.index.values, left_on=df_player_data.Team, right_index=True)
projected_points = df_player_data.Average_PPG*df_player_data.Games_Per_Week
df_player_data['Proj. Points'] = projected_points

In [10]:
qgrid.show_grid(df_player_data, remote_js=True, grid_options=week_info.grid_opts)

In [9]:
df = week_info.get_past_3_seasons_player_stats()
qgrid.show_grid(df, remote_js=True)